# 4. Over und -Underfitting auf den "Hitters"-Daten

In diesem Notebook wird eine lineare Regression auf dem schon bekannten `Hitters`
-Datensatz trainiert. Dabei werden verschiedene Formen der Regularisierung gegen Overfitting eingesetzt.

Hinweis: Da das Notebook nur Regularisierung veranschaulichen soll, ist der ML-Worflow in diesem Notebook stark vereinfacht (kein Auffüllen von N/As, kein Feature-Scaling und keine kategorischen Features).

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error 

### Daten laden und vorbereiten

Wir laden den `Hitters`-Datensatz und wollen anhand von Attributen von Baseball-Spielern deren Gehalt (Spalte `Salary`) vorhersagen.

In [2]:
df = pd.read_csv('data/hitters.csv')
df = df.dropna()
df = df.drop(["Player", "League", "Division", "NewLeague"], axis=1)
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,475.0
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,480.0
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,500.0
4,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,91.5
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,750.0


In [3]:
X = df.drop(['Salary'], axis=1)
y = df["Salary"]
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Lineare Regression

In [4]:
reg = LinearRegression()
reg.fit(X_train, y_train);

Als nächstes schauen wir auf den MSE auf den Testdaten und die trainierten Gewichte der Features:

In [5]:
pred = reg.predict(X_test)
print("Mean squared error:", mean_squared_error(y_test, pred))
print("Gewichte der Features:")
print(pd.Series(reg.coef_, index=X.columns))

sum_weights = sum(np.absolute(reg.coef_))
print("Summe der absoluten Gewichte:", sum_weights)

Mean squared error: 132925.0044529528
Gewichte der Features:
AtBat     -2.015452
Hits       7.520141
HmRun      9.312551
Runs      -1.719625
RBI       -2.340944
Walks      7.285795
Years      7.682881
CAtBat    -0.165969
CHits      0.274872
CHmRun    -0.531528
CRuns      1.164023
CRBI       0.653199
CWalks    -0.828976
PutOuts    0.198798
Assists    0.452282
Errors    -5.308274
dtype: float64
Summe der absoluten Gewichte: 47.455309856503135


### Ridge Regression

Wir trainieren nun eine Ridge-Regression um beim Training eine L2-Regularisierung zu verwenden. Dazu probieren wir zuerst den Parameter $ \alpha = 100$ als Gewichtung für den Regressionsterm aus.

In [6]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha = 100)
ridge.fit(X_train, y_train);

Wir schauen wieder auf den MSE auf den Testdaten und die Gewichte der Features:

In [7]:
pred = ridge.predict(X_test)          
print("Mean squared error:", mean_squared_error(y_test, pred))
print("Gewichte der Features:")
print(pd.Series(ridge.coef_, index = X.columns))
print("Summe der absoluten Gewichte:", sum(np.absolute(ridge.coef_)))

Mean squared error: 131901.3136464271
Gewichte der Features:
AtBat     -2.017375
Hits       7.412108
HmRun      8.636404
Runs      -1.540755
RBI       -2.105771
Walks      7.192270
Years      6.426333
CAtBat    -0.161598
CHits      0.285102
CHmRun    -0.452459
CRuns      1.134987
CRBI       0.625129
CWalks    -0.818690
PutOuts    0.199536
Assists    0.446716
Errors    -5.237216
dtype: float64
Summe der absoluten Gewichte: 44.69244920434857


Wir sehen, dass die Gewichte im Schnitt kleiner sind und dass wir den Test-MSE verringern konnten.

Testweise probieren wir einen höheren Wert für $ \alpha $ aus:

In [8]:
ridge_high = Ridge(alpha = 10**10)
ridge_high.fit(X_train, y_train)             
pred_high = ridge_high.predict(X_test)
print("Mean squared error:", mean_squared_error(y_test, pred_high))
print(pd.Series(ridge_high.coef_, index = X.columns)) 
print("Summe der absoluten Gewichte:", sum(np.absolute(ridge_high.coef_)))

Mean squared error: 281106.82640631875
AtBat      5.245567e-04
Hits       1.808956e-04
HmRun      2.583309e-05
Runs       9.702965e-05
RBI        9.134841e-05
Walks      7.854807e-05
Years      1.383971e-05
CAtBat     8.791470e-03
CHits      2.628744e-03
CHmRun     2.581522e-04
CRuns      1.357800e-03
CRBI       1.230443e-03
CWalks     9.105701e-04
PutOuts    6.469467e-04
Assists    9.068915e-05
Errors     8.876302e-08
dtype: float64
Summe der absoluten Gewichte: 0.016926954821158316


Wir sehen, dass die Gewichte jetzt alle auf nahezu Null gedrückt werden, der MSE aber dadurch riesig wird.
Die Frage ist daher, mit welchem Wert für $ \alpha $ können wir die besten Ergebnisse erzielen?

Mit Hilfe der "Cross Validation"-Klassen können wir automatisch verschiedene $\alpha$-Werte testen und bekommen den besten Wert auf den Validierungsdaten zurück.
Dazu erstellen wir im ersten Schritt eine Kandidatenmenge für $ \alpha $:

In [9]:
alphas = list(range(1, 10000))
print(f"Teste {len(alphas)} verschiedene Werte für Alpha")

Teste 9999 verschiedene Werte für Alpha


Die Klasse `RidgeCV` führt automatisch eine 5-fold Cross-Validation durch und bestimmt so den besten Parameter für $\alpha$:

In [10]:
ridgecv = RidgeCV(alphas=alphas)
ridgecv.fit(X_train, y_train)
ridgecv.alpha_ # bester Parameter

4760

Mit diesem Modell machen wir Vorhersagen auf den Testdaten und schauen uns den MSE an:

In [11]:
mse = mean_squared_error(y_test, ridgecv.predict(X_test))
print("MSE ist: " + str(mse))

MSE ist: 124328.37417267624


### Lasso Regression

Wie in der Vorlesung gesehen, minimiert Lasso-Regularisierung die Gewichte in dem es versucht einige Gewichtswerte auf Null zu bringen. Auch hier führen wir eine CV durch:

In [12]:
lassocv = LassoCV(alphas=alphas, max_iter=100000)
lassocv.fit(X_train, y_train)
mse = mean_squared_error(y_test, lassocv.predict(X_test))
print("Mean squared error:", mse)
print("Gewichte der Feature:")
pd.Series(lassocv.coef_, index=X.columns)

Mean squared error: 129381.15596025465
Gewichte der Feature:


AtBat     -0.000000
Hits       1.818999
HmRun      0.000000
Runs       0.000000
RBI        0.000000
Walks      1.791831
Years      0.000000
CAtBat    -0.241001
CHits      0.727059
CHmRun     0.000000
CRuns      0.530056
CRBI       0.416534
CWalks    -0.084698
PutOuts    0.181185
Assists    0.077650
Errors    -0.000000
dtype: float64

Wir sehen, dass einige Features (z.B. `AtBat` oder `Years`) jetzt den Gewichtswert `0` haben, d.h. sie tragen nicht mehr zur Vorhersage bei. Der MSE wird ein jedoch ein klein wenig schlechter als bei der Ridge-Regression.